# 1. Image Labelling

Now, let's dive into the process of labelling images using the Image Labeler app in MATLAB. This step-by-step guide will help you annotate the wear sites within the images effectively.

* Open MATLAB and Launch the Image Labeler App:
    * Navigate to the APPS tab.
    * Click on Image Labeler.
* Import Images:
    * In the Label Toolbar within the Image Labeler app, select Import.
    * Choose From File and select the images you downloaded earlier.
* Define Pixel Labels:
    * In the ROI Labels section, select Label.
    * Choose Pixel label and add a label name (e.g., "Wear Site").
* Annotation Process:
    * Use the Polygon tool to mark the wear areas within the images accurately.
* Save Your Progress:
    * Save the session by clicking on the save icon or using the shortcut Ctrl + S.
    * This will create an imageLabellingSession.mat file, preserving your annotations.
* Export Labelled Images:
    * Once you've finished annotating all the images, it's time to export the ground truth data.
    * In the Label Toolbar, select Export to File.
<div style="text-align: center;">
    <img src="VC_Images/Img_01.png" alt="Alt text" style="display: block; margin: 0 auto;">
</div>


# 2. Mask Generating

To convert the ground truth annotations into masks, we will apply linearly distributed thresholds to obtain the original class values. This process allows us to translate the labelled regions into binary masks, facilitating further analysis and model training.

In [27]:
import os
import cv2
import numpy as np
from PIL import Image
from skimage import io, img_as_ubyte
import random
from scipy.ndimage import rotate

In [ ]:
# Get the current working directory
current_dir = os.getcwd()

# Now create the path to the folder with Images for Converting
readDirectory = os.path.join(current_dir, 'Image_Dataset/02_Labeled_Images')
writeDirectory = os.path.join(current_dir, 'Image_Dataset/03_Converted_Masks')

In [14]:
numClasses = 2  # Number of classes used for classification

# Find different color values
img_name_list = sorted(os.listdir(readDirectory))
print('Dataset size: ', len(img_name_list))

# Find unique colors and list them
uniqueColors = set()
for img_name in img_name_list:
    img_path = os.path.join(readDirectory, img_name)
    img = cv2.imread(img_path, flags=0)  # Open image in greyscale mode
    uniqueColors |= set(np.unique(img))
print("Found Colors (unique greyscale values [0..255]): ", len(uniqueColors))

# Categorize using thresholds
minimum = min(uniqueColors)
maximum = max(uniqueColors)
print('Minimum Threshold: ' + str(minimum))
print('Maximum Threshold: ' + str(maximum))

# Generate linearly distributed thresholds
thresholds = np.linspace(start=minimum, stop=maximum + 1, num=numClasses + 1)

# Generate linearly distributed classes [0..255]
colorClasses = np.linspace(start=0, stop=255, num=numClasses)
print("New greyscale values: ", colorClasses)

# Apply thresholds on masks
for img_name in img_name_list:
    img_path = os.path.join(readDirectory, img_name)
    img = cv2.imread(img_path, flags=0)

    # Using discrete values from colorClasses as new values
    for c in range(numClasses):
        img[(thresholds[c] <= img) & (thresholds[c + 1] > img)] = colorClasses[c]

    # Save mask as .png (no compression loss)
    img_path = os.path.join(writeDirectory, img_name.split(".")[0] + ".png")
    cv2.imwrite(img_path, img)

Dataset size:  35
Found Colors (unique greyscale values [0..255]):  2
Minimum Threshold: 0
Maximum Threshold: 1
New greyscale values:  [  0. 255.]


# 3. ROI Extracting

The images are large in size (above 2000 x 2000 pixels), so it is better to extract the area that is important to us by using a cropping function. This cropping process should be applied to both the image and the respective masks.

<div style="text-align: center;">
    <img src="VC_Images/Img_02.png" alt="Alt text" style="display: block; margin: 0 auto;">
</div>

In [9]:
# Get the current working directory
Image_Dir_Inp = os.path.join(current_dir, 'Image_Dataset/01_Image_Labelling')
Image_Dir_Out = os.path.join(current_dir, 'Image_Dataset/04_Region_Of_Interest/01_Images')
Mask_Dir_Inp = os.path.join(current_dir, 'Image_Dataset/03_Converted_Masks')
Mask_Dir_Out = os.path.join(current_dir, 'Image_Dataset/04_Region_Of_Interest/02_Masks')

In [16]:
# ROI For Image    
for filename in os.listdir(Image_Dir_Inp): 
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        # Open the image
        img_path = os.path.join(Image_Dir_Inp, filename)
        img = cv2.imread(img_path)  
        imgheight = img.shape[0]
        imgwidth = img.shape[1]
        # Slicing to crop the image
        cropped_image = img[int(imgwidth/2)-1000:int(imgwidth/2)+1000, int(imgheight/2)-700:int(imgheight/2)+700]

        # Output filename with "_cropped" added
        output_filename = os.path.splitext(filename)[0] + "_cropped.png"

        # Save the cropped image to the output directory
        output_path = os.path.join(Image_Dir_Out, output_filename)
        cv2.imwrite(output_path, cropped_image)

In [15]:
# ROI For Masks   
for filename in os.listdir(Image_Dir_Inp): 
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        # Open the image
        img_path = os.path.join(Image_Dir_Inp, filename)
        img = cv2.imread(img_path)  
        imgheight = img.shape[0]
        imgwidth = img.shape[1]
        # Slicing to crop the image
        cropped_image = img[int(imgwidth/2)-1000:int(imgwidth/2)+1000, int(imgheight/2)-700:int(imgheight/2)+700]

        # Output filename with "_cropped" added
        output_filename = os.path.splitext(filename)[0] + "_cropped.png"

        # Save the cropped image to the output directory
        output_path = os.path.join(Image_Dir_Out, output_filename)
        cv2.imwrite(output_path, cropped_image)

# 4. Data Converting

It is recommended to convert the format of both the image and mask to TIFF format, which is suitable for the Convolutional Neural Network (CNN) model that will be used.

In [17]:
# Get the current working directory
ROI_Image_Dir_Inp = os.path.join(current_dir, 'Image_Dataset/04_Region_Of_Interest/01_Images')
TIFF_Image_Dir_Out = os.path.join(current_dir, 'Image_Dataset/05_Format_Converting/01_Images')
ROI_Mask_Dir_Inp = os.path.join(current_dir, 'Image_Dataset/04_Region_Of_Interest/02_Masks')
TIFF_Mask_Dir_Out = os.path.join(current_dir, 'Image_Dataset/05_Format_Converting/02_Masks')

In [20]:
# Format tif For Image   
for filename in os.listdir(ROI_Image_Dir_Inp):
    if filename.endswith(('.jpg', '.jpeg', '.png')):  # Add other image formats if needed
        # Open the image
        img_path = os.path.join(ROI_Image_Dir_Inp, filename)
        im = Image.open(img_path)
        # Save the resized image in TIFF format to the output folder
        output_path = os.path.join(TIFF_Image_Dir_Out, os.path.splitext(filename)[0] + '.tif')
        im.save(output_path, 'TIFF')
print("Conversion completed.")


Conversion completed.


In [21]:
# Format tif For Mask   
for filename in os.listdir(ROI_Mask_Dir_Inp):
    if filename.endswith(('.jpg', '.jpeg', '.png')):  # Add other image formats if needed
        # Open the image
        img_path = os.path.join(ROI_Mask_Dir_Inp, filename)
        im = Image.open(img_path)
        # Save the resized image in TIFF format to the output folder
        output_path = os.path.join(TIFF_Mask_Dir_Out, os.path.splitext(filename)[0] + '.tif')
        im.save(output_path, 'TIFF')
print("Conversion completed.")

Conversion completed.


# 4. Image Augmentation

After extracting the ROI and converting the image and mask to TIFF format, we need to increase our dataset size using augmentation techniques. Here's how you can implement this method, ensuring that the image name and the respective mask name are the same:

* Define Augmentation Parameters: Determine the augmentation techniques to apply, such as rotation, flipping, scaling, etc.

* Loop Through Images: Iterate through each image and its corresponding mask.

* Apply Augmentation: Apply the defined augmentation techniques to both the image and its mask.

* Save Augmented Images: Save the augmented images and their masks with the same names as the original images and masks.

<div style="text-align: center;">
    <img src="VC_Images/Img_03.png" alt="Alt text" style="display: block; margin: 0 auto;">
</div>

In [34]:
# Get the current working directory
images_path= os.path.join(current_dir, 'Image_Dataset/05_Format_Converting/01_Images')
masks_path = os.path.join(current_dir, 'Image_Dataset/05_Format_Converting/02_Masks')
img_augmented_path= os.path.join(current_dir, 'Image_Dataset/06_Image_Augmentation/01_Images')
msk_augmented_path= os.path.join(current_dir, 'Image_Dataset/06_Image_Augmentation/02_Masks')

In [36]:
#Define functions for each operation
def rotation(image, seed):
    random.seed(seed)
    angle= 180
    r_img = rotate(image, angle, mode='reflect', reshape=False, order=0)
    return r_img

def h_flip(image, seed):
    hflipped_img= np.fliplr(image)
    return  hflipped_img

def v_flip(image, seed):
    vflipped_img= np.flipud(image)
    return vflipped_img

def v_transl(image, seed):
    random.seed(seed)
    n_pixels = random.randint(-64,64)
    vtranslated_img = np.roll(image, n_pixels, axis=0)
    return vtranslated_img

def h_transl(image, seed):
    random.seed(seed)
    n_pixels = random.randint(-64,64)
    htranslated_img = np.roll(image, n_pixels, axis=1)
    return htranslated_img

transformations = {'rt': rotation,
                      'hf': h_flip, 
                      'vf': v_flip
                 }                #use dictionary to store names of functions

In [ ]:
# to store paths of images from folder
images=[] 
images_name=[]
masks=[]
masks_name=[]

for im in os.listdir(images_path):  # read image name from folder and append its path into "images" array     
    images.append(os.path.join(images_path,im))
    images_name.append(im) 

for msk in os.listdir(masks_path):  # read mask name from folder and append its path into "masks" array     
    masks.append(os.path.join(masks_path,msk))
    masks_name.append(msk) 

In [37]:
for i in range(len(images)): 
    image = images[i]
    mask = masks[i]
    original_image = io.imread(image)
    original_mask = io.imread(mask)
    transformed_image = None
    transformed_mask = None
    
    for n in range(len(transformations)):
        key = list(transformations)[n] #choosing method to call
        seed = random.randint(1,100)  #Generate seed to supply transformation functions. 
        # Generating the augmented data
        transformed_image = transformations[key](original_image,seed)
        transformed_mask = transformations[key](original_mask,seed)
        # Saving the augmented data
        new_image_path= "%s/%s_aug_%s.tif" %(img_augmented_path, images_name[i][:-4], key)
        new_mask_path= "%s/%s_aug_%s.tif" %(msk_augmented_path, masks_name[i][:-4], key)
        io.imsave(new_image_path, transformed_image)
        io.imsave(new_mask_path, transformed_mask)

C:\Users\karimi\AppData\Local\Temp\ipykernel_24348\3534118877.py:25: UserWarning: d:\02-Project\01-Wear Detection\09-PROKI\BP_Tool_Wear_Measurement\Image_Dataset/06_Image_Augmentation/02_Masks/image0000885_cropped_aug_rt.tif is a low contrast image
  io.imsave(new_mask_path, transformed_mask)
C:\Users\karimi\AppData\Local\Temp\ipykernel_24348\3534118877.py:25: UserWarning: d:\02-Project\01-Wear Detection\09-PROKI\BP_Tool_Wear_Measurement\Image_Dataset/06_Image_Augmentation/02_Masks/image0000885_cropped_aug_hf.tif is a low contrast image
  io.imsave(new_mask_path, transformed_mask)
C:\Users\karimi\AppData\Local\Temp\ipykernel_24348\3534118877.py:25: UserWarning: d:\02-Project\01-Wear Detection\09-PROKI\BP_Tool_Wear_Measurement\Image_Dataset/06_Image_Augmentation/02_Masks/image0000885_cropped_aug_vf.tif is a low contrast image
  io.imsave(new_mask_path, transformed_mask)
C:\Users\karimi\AppData\Local\Temp\ipykernel_24348\3534118877.py:25: UserWarning: d:\02-Project\01-Wear Detection\09-P